In [78]:
for year in range(1993, 2021):
    for mounth in ['06', '11']:
        wget_str = f'https://top500.org/lists/top500/{year}/{mounth}/download/TOP500_{year}{mounth}.xls'
        if year == 2020 and mounth == '06':
            print(wget_str + 'x')
        elif year == 2020 and mounth == '11':
            # закончились данные
            break
        else:
            print(wget_str)

https://top500.org/lists/top500/1993/06/download/TOP500_199306.xls
https://top500.org/lists/top500/1993/11/download/TOP500_199311.xls
https://top500.org/lists/top500/1994/06/download/TOP500_199406.xls
https://top500.org/lists/top500/1994/11/download/TOP500_199411.xls
https://top500.org/lists/top500/1995/06/download/TOP500_199506.xls
https://top500.org/lists/top500/1995/11/download/TOP500_199511.xls
https://top500.org/lists/top500/1996/06/download/TOP500_199606.xls
https://top500.org/lists/top500/1996/11/download/TOP500_199611.xls
https://top500.org/lists/top500/1997/06/download/TOP500_199706.xls
https://top500.org/lists/top500/1997/11/download/TOP500_199711.xls
https://top500.org/lists/top500/1998/06/download/TOP500_199806.xls
https://top500.org/lists/top500/1998/11/download/TOP500_199811.xls
https://top500.org/lists/top500/1999/06/download/TOP500_199906.xls
https://top500.org/lists/top500/1999/11/download/TOP500_199911.xls
https://top500.org/lists/top500/2000/06/download/TOP500_200006

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
data_list = sorted(os.listdir('data'))
data_list[:5], data_list[-5:]

(['TOP500_199306.xls',
  'TOP500_199311.xls',
  'TOP500_199406.xls',
  'TOP500_199411.xls',
  'TOP500_199506.xls'],
 ['TOP500_201806.xls',
  'TOP500_201811.xls',
  'TOP500_201906.xls',
  'TOP500_201911.xls',
  'TOP500_202006.xlsx'])

In [4]:
data_1 = []


for i in range(30):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=1, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[0]))
    
for i in range(30, 37):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[0]))
    
for i in range(37, 48):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].iloc[0]))
    
for i in range(48, 55):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].iloc[0]*1000))
    

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [5]:
data_500 = []


for i in range(30):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=1, index_col=0)
    print(i, data_list[i])
    print(data_list[i][7:11], df['RMax'].iloc[-1])
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[-1]))
    
for i in range(30, 37):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    print(i, data_list[i])
    print(data_list[i][7:11], df['RMax'].iloc[-1])
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[-1]))
    
for i in range(37, 48):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    print(i, data_list[i])
    print(data_list[i][7:11], df['Rmax'].iloc[-1])
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].iloc[-1]))
    
for i in range(48, 55):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    print(i, data_list[i])
    print(data_list[i][7:11], df['Rmax [TFlop/s]'].iloc[-1])
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].iloc[-1]*1000))
    

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
0 TOP500_199306.xls
1993 0.422
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
1 TOP500_199311.xls
1993 0.472
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
2 TOP500_199406.xls
1994 0.822
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
3 TOP500_199411.xls
1994 1.114
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
4 TOP500_199506.xls
1995 1.955
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
5 TOP500_199511.xls
1995 2.489
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
6 TOP500_199606.xls
1996 3.306
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
7 TOP500_199611.xls
1996 4.62
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
8 TOP500_199706.xls
1997 7.67
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is no

In [6]:
data_sum = []


for i in range(30):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=1, index_col=0)
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].sum()))
    
for i in range(30, 37):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].sum()))
    
for i in range(37, 48):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].sum()))
    
for i in range(48, 55):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].sum()*1000))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [7]:
parse_data = {
    'data': [i[0] for i in data_1],
    '#1 GFlop/s': [i[1] for i in data_1],
    '#500 GFlop/s': [i[1] for i in data_500],
    'Sum GFlop/s': [i[1] for i in data_sum],
}

In [8]:
df = pd.DataFrame(parse_data)
df.head()

,data,#1 GFlop/s,#500 GFlop/s,Sum GFlop/s
0,06/1993,59.7,0.422,1122.845
1,11/1993,124.0,0.472,1466.210
2,06/1994,143.4,0.822,2233.882
3,11/1994,170.0,1.114,2622.338
4,06/1995,170.0,1.955,3927.029


In [9]:
df.to_csv('data_top500_hpc.csv')